In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ddos-sdn-dataset/dataset_sdn.csv


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [7]:
df = pd.read_csv("/kaggle/input/ddos-sdn-dataset/dataset_sdn.csv")
df.head()


,dt,switch,src,dst,pktcount,bytecount,dur,dur_nsec,tot_dur,flows,...,pktrate,Pairflow,Protocol,port_no,tx_bytes,rx_bytes,tx_kbps,rx_kbps,tot_kbps,label
0,11425,1,10.0.0.1,10.0.0.8,45304,48294064,100,716000000,1.010000e+11,3,...,451,0,UDP,3,143928631,3917,0,0.0,0.0,0
1,11605,1,10.0.0.1,10.0.0.8,126395,134737070,280,734000000,2.810000e+11,2,...,451,0,UDP,4,3842,3520,0,0.0,0.0,0
2,11425,1,10.0.0.2,10.0.0.8,90333,96294978,200,744000000,2.010000e+11,3,...,451,0,UDP,1,3795,1242,0,0.0,0.0,0
3,11425,1,10.0.0.2,10.0.0.8,90333,96294978,200,744000000,2.010000e+11,3,...,451,0,UDP,2,3688,1492,0,0.0,0.0,0
4,11425,1,10.0.0.2,10.0.0.8,90333,96294978,200,744000000,2.010000e+11,3,...,451,0,UDP,3,3413,3665,0,0.0,0.0,0


In [8]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104345 entries, 0 to 104344
Data columns (total 23 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   dt           104345 non-null  int64  
 1   switch       104345 non-null  int64  
 2   src          104345 non-null  object 
 3   dst          104345 non-null  object 
 4   pktcount     104345 non-null  int64  
 5   bytecount    104345 non-null  int64  
 6   dur          104345 non-null  int64  
 7   dur_nsec     104345 non-null  int64  
 8   tot_dur      104345 non-null  float64
 9   flows        104345 non-null  int64  
 10  packetins    104345 non-null  int64  
 11  pktperflow   104345 non-null  int64  
 12  byteperflow  104345 non-null  int64  
 13  pktrate      104345 non-null  int64  
 14  Pairflow     104345 non-null  int64  
 15  Protocol     104345 non-null  object 
 16  port_no      104345 non-null  int64  
 17  tx_bytes     104345 non-null  int64  
 18  rx_bytes     104345 non-

,dt,switch,pktcount,bytecount,dur,dur_nsec,tot_dur,flows,packetins,pktperflow,byteperflow,pktrate,Pairflow,port_no,tx_bytes,rx_bytes,tx_kbps,rx_kbps,tot_kbps,label
count,104345.000000,104345.000000,104345.000000,1.043450e+05,104345.000000,1.043450e+05,1.043450e+05,104345.000000,104345.000000,104345.000000,1.043450e+05,104345.000000,104345.000000,104345.000000,1.043450e+05,1.043450e+05,104345.000000,103839.000000,103839.000000,104345.000000
mean,17927.514169,4.214260,52860.954746,3.818660e+07,321.497398,4.613880e+08,3.218865e+11,5.654234,5200.383468,6381.715291,4.716150e+06,212.210676,0.600987,2.331094,9.325264e+07,9.328039e+07,998.899756,1003.811420,2007.578742,0.390857
std,11977.642655,1.956327,52023.241460,4.877748e+07,283.518232,2.770019e+08,2.834029e+11,2.950036,5257.001450,7404.777808,7.560116e+06,246.855123,0.489698,1.084333,1.519380e+08,1.330004e+08,2423.471618,2054.887034,3144.437173,0.487945
min,2488.000000,1.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,2.000000,4.000000,-130933.000000,-1.464426e+08,-4365.000000,0.000000,1.000000,2.527000e+03,8.560000e+02,0.000000,0.000000,0.000000,0.000000
25%,7098.000000,3.000000,808.000000,7.957600e+04,127.000000,2.340000e+08,1.270000e+11,3.000000,1943.000000,29.000000,2.842000e+03,0.000000,0.000000,1.000000,4.743000e+03,3.539000e+03,0.000000,0.000000,0.000000,0.000000
50%,11905.000000,4.000000,42828.000000,6.471930e+06,251.000000,4.180000e+08,2.520000e+11,5.000000,3024.000000,8305.000000,5.521680e+05,276.000000,1.000000,2.000000,4.219610e+06,1.338339e+07,0.000000,0.000000,4.000000,0.000000
75%,29952.000000,5.000000,94796.000000,7.620354e+07,412.000000,7.030000e+08,4.130000e+11,7.000000,7462.000000,10017.000000,9.728112e+06,333.000000,1.000000,3.000000,1.356398e+08,1.439277e+08,251.000000,557.000000,3838.000000,1.000000
max,42935.000000,10.000000,260006.000000,1.471280e+08,1881.000000,9.990000e+08,1.880000e+12,17.000000,25224.000000,19190.000000,1.495387e+07,639.000000,1.000000,5.000000,1.269982e+09,9.905962e+08,20580.000000,16577.000000,20580.000000,1.000000


In [9]:
X = df.drop("label", axis=1)
y = df["label"]


In [10]:
X = X.drop(["src", "dst"], axis=1)


In [11]:
X["Protocol"].value_counts()


Protocol
ICMP    41321
UDP     33588
TCP     29436
Name: count, dtype: int64

In [12]:
X = pd.get_dummies(X, columns=["Protocol"], drop_first=True)


In [13]:
X.isnull().sum()


dt                0
switch            0
pktcount          0
bytecount         0
dur               0
dur_nsec          0
tot_dur           0
flows             0
packetins         0
pktperflow        0
byteperflow       0
pktrate           0
Pairflow          0
port_no           0
tx_bytes          0
rx_bytes          0
tx_kbps           0
rx_kbps         506
tot_kbps        506
Protocol_TCP      0
Protocol_UDP      0
dtype: int64

In [14]:
X = X.fillna(X.mean())


In [15]:
X.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104345 entries, 0 to 104344
Data columns (total 21 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   dt            104345 non-null  int64  
 1   switch        104345 non-null  int64  
 2   pktcount      104345 non-null  int64  
 3   bytecount     104345 non-null  int64  
 4   dur           104345 non-null  int64  
 5   dur_nsec      104345 non-null  int64  
 6   tot_dur       104345 non-null  float64
 7   flows         104345 non-null  int64  
 8   packetins     104345 non-null  int64  
 9   pktperflow    104345 non-null  int64  
 10  byteperflow   104345 non-null  int64  
 11  pktrate       104345 non-null  int64  
 12  Pairflow      104345 non-null  int64  
 13  port_no       104345 non-null  int64  
 14  tx_bytes      104345 non-null  int64  
 15  rx_bytes      104345 non-null  int64  
 16  tx_kbps       104345 non-null  int64  
 17  rx_kbps       104345 non-null  float64
 18  tot_

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [17]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)


RandomForestClassifier(n_jobs=-1, random_state=42)

In [18]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = rf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
confusion_matrix(y_test, y_pred)


Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12712
           1       1.00      1.00      1.00      8157

    accuracy                           1.00     20869
   macro avg       1.00      1.00      1.00     20869
weighted avg       1.00      1.00      1.00     20869



array([[12712,     0],
       [    0,  8157]])

In [19]:
import pandas as pd

importances = rf.feature_importances_

fi = pd.DataFrame({
    "feature": X.columns,
    "importance": importances
}).sort_values(by="importance", ascending=False)

fi.head(10)


,feature,importance
10,byteperflow,0.189543
3,bytecount,0.152462
9,pktperflow,0.142564
11,pktrate,0.129627
2,pktcount,0.115906
8,packetins,0.093404
4,dur,0.036948
0,dt,0.035686
6,tot_dur,0.034614
19,Protocol_TCP,0.020469


In [20]:
top_feature = fi.iloc[0]["feature"]
print("Dropped feature:", top_feature)

X_leak_test = X.drop(top_feature, axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X_leak_test, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print(classification_report(y_test, y_pred))


Dropped feature: byteperflow
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12712
           1       1.00      1.00      1.00      8157

    accuracy                           1.00     20869
   macro avg       1.00      1.00      1.00     20869
weighted avg       1.00      1.00      1.00     20869



In [21]:
drop_cols = [
    "pktcount", "bytecount", "pktperflow",
    "byteperflow", "pktrate", "packetins"
]

X_hard = X.drop(drop_cols, axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X_hard, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.98      0.98      0.98     12712
           1       0.98      0.97      0.97      8157

    accuracy                           0.98     20869
   macro avg       0.98      0.98      0.98     20869
weighted avg       0.98      0.98      0.98     20869



In [22]:
print("Train acc:", rf.score(X_train, y_train))
print("Test acc :", rf.score(X_test, y_test))


Train acc: 0.9995208203555513
Test acc : 0.9786765058220327


In [23]:
confusion_matrix(y_test, y_pred)


array([[12514,   198],
       [  247,  7910]])

In [24]:
rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=12,
    min_samples_leaf=5,
    min_samples_split=10,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

print("Train acc:", rf.score(X_train, y_train))
print("Test acc :", rf.score(X_test, y_test))
confusion_matrix(y_test, y_pred)


Train acc: 0.9347477119171977
Test acc : 0.9245292059993292


array([[12514,   198],
       [  247,  7910]])

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_hard, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [26]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)


In [27]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)


(83476, 15)
(20869, 15)


In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),

    Dense(32, activation="relu"),
    Dropout(0.3),

    Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)


2025-12-22 04:46:12.518387: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766378772.767673      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766378772.839977      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766378773.452587      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766378773.452650      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766378773.452653      55 computation_placer.cc:177] computation placer alr

In [29]:
history = model.fit(
    X_train_scaled, y_train,
    epochs=20,
    batch_size=256,
    validation_split=0.2,
    verbose=1
)


Epoch 1/20
261/261 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6304 - loss: 0.6419 - val_accuracy: 0.7169 - val_loss: 0.5342
Epoch 2/20
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7005 - loss: 0.5474 - val_accuracy: 0.7524 - val_loss: 0.4980
Epoch 3/20
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7268 - loss: 0.5199 - val_accuracy: 0.7604 - val_loss: 0.4812
Epoch 4/20
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7372 - loss: 0.5048 - val_accuracy: 0.7733 - val_loss: 0.4692
Epoch 5/20
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7510 - loss: 0.4892 - val_accuracy: 0.7789 - val_loss: 0.4595
Epoch 6/20
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7562 - loss: 0.4822 - val_accuracy: 0.7772 - val_loss: 0.4543
Epoch 7/20
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7615 - loss: 0.4753 - val_accuracy: 0.7858 - val_loss: 0.4428
Epoch 8/20
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7638 - loss: 0.4698 - val_accuracy: 0.

In [30]:
test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=0)
print("ANN Test accuracy:", test_acc)


ANN Test accuracy: 0.8166179656982422


In [31]:
import numpy as np

y_pred_prob = model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int)


653/653 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [32]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_pred)
print(cm)


[[11349  1363]
 [ 2464  5693]]


In [33]:
print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

           0       0.82      0.89      0.86     12712
           1       0.81      0.70      0.75      8157

    accuracy                           0.82     20869
   macro avg       0.81      0.80      0.80     20869
weighted avg       0.82      0.82      0.81     20869



In [34]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report


In [35]:

xgb = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

xgb.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=-1,
              num_parallel_tree=None, ...)

In [36]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = xgb.predict(X_test)

print("XGB Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


XGB Accuracy: 0.9841870717331928
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     12712
           1       0.98      0.98      0.98      8157

    accuracy                           0.98     20869
   macro avg       0.98      0.98      0.98     20869
weighted avg       0.98      0.98      0.98     20869



In [37]:
from sklearn.metrics import confusion_matrix

cm_xgb = confusion_matrix(y_test, y_pred)
print(cm_xgb)


[[12563   149]
 [  181  7976]]


In [38]:
import numpy as np

def print_metrics(name, cm):
    TN, FP = cm[0]
    FN, TP = cm[1]
    total = TN + FP + FN + TP

    accuracy  = (TP + TN) / total
    precision = TP / (TP + FP)
    recall    = TP / (TP + FN)          # TPR
    f1        = 2 * (precision * recall) / (precision + recall)
    fpr       = FP / (FP + TN)
    fnr       = FN / (FN + TP)
    tnr       = TN / (TN + FP)

    print(f"\n{'='*60}")
    print(f"{name}")
    print(f"{'='*60}")
    print("Confusion Matrix:")
    print(cm)
    print("\nMetrics:")
    print(f"Accuracy               : {accuracy:.4f}   ((TP+TN)/(TP+TN+FP+FN))")
    print(f"Precision (Attack)     : {precision:.4f}  (TP/(TP+FP))")
    print(f"Recall / TPR (Attack)  : {recall:.4f}     (TP/(TP+FN))")
    print(f"F1-Score               : {f1:.4f}         (2*(P*R)/(P+R))")
    print(f"False Positive Rate    : {fpr:.4f}        (FP/(FP+TN))")
    print(f"False Negative Rate    : {fnr:.4f}        (FN/(FN+TP))")
    print(f"Specificity / TNR      : {tnr:.4f}        (TN/(TN+FP))")

# =========================
# CONFUSION MATRICES
# =========================

cm_rf  = np.array([[12514, 198],
                   [  247, 7910]])

cm_ann = np.array([[11349, 1363],
                   [ 2464, 5693]])

cm_xgb = np.array([[12563, 149],
                   [  181, 7976]])

# =========================
# PRINT ALL
# =========================

print_metrics("Random Forest (RF)", cm_rf)
print_metrics("Artificial Neural Network (ANN)", cm_ann)
print_metrics("XGBoost (XGB)", cm_xgb)



Random Forest (RF)
Confusion Matrix:
[[12514   198]
 [  247  7910]]

Metrics:
Accuracy               : 0.9787   ((TP+TN)/(TP+TN+FP+FN))
Precision (Attack)     : 0.9756  (TP/(TP+FP))
Recall / TPR (Attack)  : 0.9697     (TP/(TP+FN))
F1-Score               : 0.9726         (2*(P*R)/(P+R))
False Positive Rate    : 0.0156        (FP/(FP+TN))
False Negative Rate    : 0.0303        (FN/(FN+TP))
Specificity / TNR      : 0.9844        (TN/(TN+FP))

Artificial Neural Network (ANN)
Confusion Matrix:
[[11349  1363]
 [ 2464  5693]]

Metrics:
Accuracy               : 0.8166   ((TP+TN)/(TP+TN+FP+FN))
Precision (Attack)     : 0.8068  (TP/(TP+FP))
Recall / TPR (Attack)  : 0.6979     (TP/(TP+FN))
F1-Score               : 0.7484         (2*(P*R)/(P+R))
False Positive Rate    : 0.1072        (FP/(FP+TN))
False Negative Rate    : 0.3021        (FN/(FN+TP))
Specificity / TNR      : 0.8928        (TN/(TN+FP))

XGBoost (XGB)
Confusion Matrix:
[[12563   149]
 [  181  7976]]

Metrics:
Accuracy               : 

## Final Conclusion

- Random Forest and XGBoost achieved high detection performance.
- XGBoost provided the lowest False Negative Rate, making it
  the most suitable model for SDN-based IDS.
- ANN showed significantly lower recall for attack traffic
  and was used only for comparison purposes.
